# Real Time Eye Gaze Tracking with Kinect


In [1]:
from PIL import Image
import cv2
import numpy as np
import os
import json
import pandas as pd
import bson
from tqdm import tqdm
from pprint import pprint
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline

import sys
sys.path.append('../')

from brspy.export import GazesExport
from brspy.export import MimicsExport
from brspy.export import JointsExport
from brspy.export import JointOrientationsExport
from brspy.export import FacePropertiesExport

from brspy.reader import Session
from utils import *

from devices import Camera
from devices import Device

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

from pyquaternion import Quaternion

## Configuration

In [2]:
markers_path = r'./markers.json'
datasets_path = r"./datasets/07_08_18"
# datasets_path = r"D:/datasets-calibration/07_08_18"
sess_path = os.path.join(datasets_path, '1533638926')
cam_data_path = r'./cam_data.json'

with open(markers_path, 'r') as f:
    markers_db = json.load(f)
    
with open(cam_data_path, 'r') as f:
    cam_data = json.load(f)
    
class HDFace:
    
    LefteyeInnercorner = '210'
    LefteyeOutercorner = '469'
    LefteyeMidtop = '241'
    LefteyeMidbottom = '1104'
    RighteyeInnercorner = '843'
    RighteyeOutercorner = '1117'
    RighteyeMidtop = '731'
    RighteyeMidbottom = '1090'
    LefteyebrowInner = '346'
    LefteyebrowOuter = '140'
    LefteyebrowCenter = '222'
    RighteyebrowInner = '803'
    RighteyebrowOuter = '758'
    RighteyebrowCenter = '849'
    MouthLeftcorner = '91'
    MouthRightcorner = '687'
    MouthUpperlipMidtop = '19'
    MouthUpperlipMidbottom = '1072'
    MouthLowerlipMidtop = '10'
    MouthLowerlipMidbottom = '8'
    NoseTip = '18'
    NoseBottom = '14'
    NoseBottomleft = '156'
    NoseBottomright = '783'
    NoseTop = '24'
    NoseTopleft = '151'
    NoseTopright = '772'
    ForeheadCenter = '28'
    LeftcheekCenter = '412'
    RightcheekCenter = '933'
    Leftcheekbone = '458'
    Rightcheekbone = '674'
    ChinCenter = '4'
    LowerjawLeftend = '1307'
    LowerjawRightend = '1327'
    
    @classmethod
    def items(cls):
        return {attr_name: getattr(cls, attr_name)
                for attr_name in dir(cls) if not attr_name.startswith('__') and not callable(getattr(cls, attr_name))}
    
    @classmethod
    def values(cls):
        return [value for key, value in cls.items().items()]
    
    @classmethod
    def keys(cls):
        return [key for key, value in cls.items().items()]
    
EyeRightCircleIdx = [776,  777,  846,  843,  1098, 
                     1097,  1095,  1096,  1096,  1091, 
                     1090,  1092,  1099,  1094,  1093, 
                     1100,  1101,  1102,  1117,  1071, 
                     1012,  992,  987,  752,  749, 
                     876,  733,  731,  728]
EyeLeftCircleIdx = [210, 316, 187, 153, 121,
                    241, 244, 238, 137, 211,
                    188, 287, 440, 1116, 469,
                    1115, 1114, 1113, 1107, 1106,
                    1112, 1105, 1104, 1103, 1108,
                    1109, 1111, 1110]

hd_all = list(map(str, range(0, 1347)))

avg_eye_radius = 0.007462746808244293

### Utils

In [3]:
from scipy.optimize import minimize


class Intersection:
    
    def __init__(self, cam_point, eye_radius, alpha=0.0001, tol=None):
        
        # model parameters
        self.eye_radius = eye_radius
        self.alpha = alpha
        self.tol = tol
        
        # points in 3d
        self.cam_point = cam_point
        self.eye_center = None
        self.ray_point = None
        
        
    def cost(self, point):
        
        # point must be on sphere surface
        sphere_cost = np.abs(np.sum((point - self.eye_center) ** 2) - (self.eye_radius ** 2))
        
        # point must be on line
        line_eq_components = (point - self.cam_point) / (self.ray_point - self.cam_point)
#         print(sphere_cost)
        line_cost = np.sum(np.abs(np.diff((point - self.cam_point) / (self.ray_point - self.cam_point))))
        
        # to prevent 2 point intersection ambiguity
        regularization = np.linalg.norm(point)
        
        # sum of all costs
        return sphere_cost ** 2 + line_cost ** 2 + sphere_cost * line_cost + self.alpha * regularization
    
    def find(self, eye_center, ray_point, verbose=True):
        
        # assign points
        self.eye_center = eye_center
        self.ray_point = ray_point
        
        # try minimize
        result = minimize(self.cost, x0=self.eye_center - self.ray_point, tol=self.tol)
        
        # print message
        if verbose:
            print(f'Cost: {result.fun:.2e}. ', result.message, sep='')
            
        return result.x
    
    def np_find(self, arr):
        return self.find(eye_center=arr[:3], ray_point=arr[3:], verbose=False)
    
    
def add_dummy_z(arr_2d):
    return np.column_stack((arr_2d, np.full(arr_2d.shape[0], 1))) 

def to_tuple(dct):
    return tuple(int(item) for key, item in dct.items())

def extract_rectangle(img, pt0, pt1):
    return img[pt0[1]:pt1[1], pt0[0]:pt1[0]]

def dict_to_np(dict_data):
    return np.array([value for axis, value in dict_data.items()]).reshape(-1, len(dict_data))

def add_dummy_z(arr_2d):
    if isinstance(arr_2d, dict):
        arr_2d = dict_to_np(arr_2d)
    return np.column_stack((arr_2d, np.full(arr_2d.shape[0], 1))) 

### Cameras 

In [4]:
for cam_name, data_dict in cam_data.items():
    Camera(cam_name, **data_dict)
    
KinectColor = Camera.get('KinectColor')
WebCamera = Camera.get('WebCamera')
InfraredCamera = Camera.get('InfraredCamera')
KinectInfrared = Camera.get('KinectInfrared')

### write markers to session

In [5]:
def to_json(vector):
    return {axis: value for axis, value in zip('XYZ', vector)}

# choose only wall 0
markers_wall_0 = {key: value for key, value in markers_db.items() if key.split('_')[1] == '0'}
pprint(markers_wall_0)

markers_save = os.path.join(sess_path, 'DataSource', 'cam_101')

try:
    os.mkdir(markers_save)
except:
    pass

counter = 0
for i in range(5):
    for j in range(8):
        marker = markers_wall_0.get(f'wall_{0}_dot_{j+1}')
        if marker:
            file_name = os.path.join(markers_save, str(counter).rjust(5, '0') + '.txt')
            with open(file_name, 'w') as f:
                json.dump(to_json(marker), f)
            counter += 1

{'wall_0_dot_1': [1.2878455803398767, -1.4462872524020634, -2.0433796750522086],
 'wall_0_dot_2': [0.3235454220662274, -1.4403578997805284, -2.0905758046738723],
 'wall_0_dot_3': [-0.4954355512992749,
                  -1.4353220953790173,
                  -2.1306595191117434],
 'wall_0_dot_4': [-1.5241555544291134,
                  -1.4289966337527293,
                  -2.1810085750519956],
 'wall_0_dot_5': [1.2919249781934634, -0.4471771977434076, -2.0018112330072593],
 'wall_0_dot_6': [0.32762481991981407,
                  -0.44124784512187254,
                  -2.0490073626289234],
 'wall_0_dot_7': [-0.4913561534456882, -0.4362120407203617, -2.089091077066794],
 'wall_0_dot_8': [-1.5200761565755267,
                  -0.42988657909407363,
                  -2.1394401330070463]}


## Create session

In [6]:
sess = Session(sess_path)

avoid = [
#     'KinectDepth',
#     'KinectInfrared',
#     'KinectBodyIndex',
#     'KinectColor',
#     'KinectBody',
#     'KinectFaceVertices',
#     'GazeEstimation',
#     'WebCamera',
#     'InfraredCamera',
#     'Markers'
]

sess.remove_devices(*avoid)

### analyse snapshot

In [7]:
def quaternion_to_rotation(quaternion):
    """
    Converts angle-axis to quaternion
    :param quaternion: dict {'X': , 'Y': , 'Z': , 'W': }
    :return: angle-axis rotation vector
    """
    if isinstance(quaternion, dict):
        quaternion = dict_to_np(quaternion).flatten()
        
    assert quaternion.ndim == 1
    assert quaternion.shape == (4,)
    
    t = np.sqrt(1 - quaternion[-1] ** 2)
    
    if t:
        return (quaternion[:3] / t)
    else:
        return np.zeros((3,))

def estimate_face_points_3d(snapshot, camera, cam_resolution_x=1296):
    
    pipe_left = make_pipeline(PolynomialFeatures(degree=9), Ridge(1e-9))
    pipe_right = make_pipeline(PolynomialFeatures(degree=9), Ridge(1e-9))
    
    # get face points
    face_points = np.array([dict_to_np(snapshot.KinectFaceVertices[idx]).flatten() for idx in hd_all])
    face_points = face_points * flip_array
    
    # get eye circle points
    EyeLeftCircle = InfraredCamera.vectors_to_self(face_points[EyeLeftCircleIdx])
    EyeRightCircle = InfraredCamera.vectors_to_self(face_points[EyeRightCircleIdx])
    
    # get pupil center on 2D image
    pupil_left_2d = dict_to_np(snapshot.GazeEstimation['pupilCenterLeft'])
    pupil_right_2d = dict_to_np(snapshot.GazeEstimation['pupilCenterRight'])
#     pupil_left_2d = dict_to_np(snapshot.ManualPupils['pupilCenterLeft'])
#     pupil_right_2d = dict_to_np(snapshot.ManualPupils['pupilCenterRight'])
    
    # get roi points of eye
    roi_left = np.array([dict_to_np(roi_pt)[0] for roi_pt in snapshot.GazeEstimation['eyeRoiLeft']])
    roi_right = np.array([dict_to_np(roi_pt)[0] for roi_pt in snapshot.GazeEstimation['eyeRoiRight']])
    
    # flip x-coordinate of 2D coordinates because of mirrored image
    pupil_left_2d[0, 0] = cam_resolution_x - pupil_left_2d[0, 0]
    pupil_right_2d[0, 0] = cam_resolution_x - pupil_right_2d[0, 0]
    roi_left[:, 0] = cam_resolution_x - roi_left[:, 0]
    roi_right[:, 0] = cam_resolution_x - roi_right[:, 0]
    
    # calculate relative position of pupil center on 2D image
    relative_pupil_left_2d = (pupil_left_2d - roi_left[0]) / np.diff(roi_left, axis=0)
    relative_pupil_right_2d = (pupil_right_2d - roi_right[0]) / np.diff(roi_right, axis=0)
    relative_pupil_left_2d[0, 0] = 1 - relative_pupil_left_2d[0, 0] 
    relative_pupil_right_2d[0, 0] = 1 - relative_pupil_right_2d[0, 0] 
    
    # estimate eye surface
    left_XY = EyeLeftCircle[:, :2]
    left_Z = EyeLeftCircle[:, 2]
    right_XY = EyeRightCircle[:, :2]
    right_Z = EyeRightCircle[:, 2]
    
    pipe_left.fit(left_XY, left_Z)
    pipe_right.fit(right_XY, right_Z)
    
    # get eye corners, top and mid points
    # and transit to Interested Camera coordinate system
    left_inner = camera.vectors_to_self(face_points[int(HDFace.LefteyeInnercorner)].reshape(1, 3)).flatten()
    left_outer = camera.vectors_to_self(face_points[int(HDFace.LefteyeOutercorner)].reshape(1, 3)).flatten()
    left_midtop = camera.vectors_to_self(face_points[int(HDFace.LefteyeMidtop)].reshape(1, 3)).flatten()
    left_midbot = camera.vectors_to_self(face_points[int(HDFace.LefteyeMidbottom)].reshape(1, 3)).flatten()
    
    right_inner = camera.vectors_to_self(face_points[int(HDFace.RighteyeInnercorner)].reshape(1, 3)).flatten()
    right_outer = camera.vectors_to_self(face_points[int(HDFace.RighteyeOutercorner)].reshape(1, 3)).flatten()
    right_midtop = camera.vectors_to_self(face_points[int(HDFace.RighteyeMidtop)].reshape(1, 3)).flatten()
    right_midbot = camera.vectors_to_self(face_points[int(HDFace.RighteyeMidbottom)].reshape(1, 3)).flatten()
    
    # get roi points X and Y in 3D in Interested Camera coordinate system
    left_top = np.array([left_inner[0], left_midtop[1]])
    left_bot = np.array([left_outer[0], left_midbot[1]])
    
    right_bot = np.array([right_inner[0], right_midbot[1]])
    right_top = np.array([right_outer[0], right_midtop[1]])
    
    # get size of roi 3 roi
    left_size_xy = np.abs(left_top - left_bot)
    right_size_xy = np.abs(right_top - right_bot)

    # estimate X and Y of 3D pupilcenter
    pupil_left_XY = left_top + relative_pupil_left_2d * left_size_xy
    pupil_right_XY = right_top + relative_pupil_right_2d * right_size_xy
    
    # estimate Z of 3D pupil_center
    pupil_left_Z = pipe_left.predict(pupil_left_XY)
    pupil_left_XYZ = np.column_stack((pupil_left_XY, pupil_left_Z))
    
    pupil_right_Z = pipe_right.predict(pupil_right_XY)
    pupil_right_XYZ = np.column_stack((pupil_right_XY, pupil_right_Z))
    
    # return pupil_center to origin coordinate system
    pupil_left_center = camera.vectors_to_origin(pupil_left_XYZ)
    pupil_right_center = camera.vectors_to_origin(pupil_right_XYZ)
    
    return face_points, pupil_left_center, pupil_right_center

def vectors_to(vectors, rotations, translations, to='self'):
    
    arr = zip(vectors, rotations, translations)

    if to is 'self':
        return np.array([(np.linalg.inv(cv2.Rodrigues(R)[0]) @ (V - T).reshape(3, 1)).flatten() for V, R, T in arr])
    elif to is 'origin':
        return np.array([(cv2.Rodrigues(R)[0] @ V.reshape(3, 1) + T.reshape(3, 1)).flatten() for V, R, T in arr])

In [8]:
markers = []
left_pupil_centers = []
right_pupil_centers = []
rotations, translations = [], []
left_gazes_true = []
left_gazes_true_person = []
right_gazes_true = []
right_gazes_true_person = []
face_points_array = []

left_eye_centers = []
right_eye_centers = []

pipe_left = make_pipeline(PolynomialFeatures(degree=9), Ridge(1e-9))
pipe_right = make_pipeline(PolynomialFeatures(degree=9), Ridge(1e-9))

frames = []

flip_array = np.array([-1, -1, 1])

for snapshot in sess.snapshots_iterate(40, verbose=True):
    
    # get pupil and face points
    face_points, left_pupil_center, right_pupil_center = estimate_face_points_3d(snapshot, InfraredCamera)
    
    # get translation and rotation of head
    translation = dict_to_np(snapshot.KinectBody['0']['Joints']['Head']['Position']).flatten() * flip_array
#     rotation = quaternion_to_rotation(snapshot.KinectFace[0]['FaceRotationQuaternion']) #* np.array([1, 1, 1])
    q = snapshot.KinectFace[0]['FaceRotationQuaternion']
    q = Quaternion(q['W'], q['X'], q['Y'], q['Z'])
    rotation = cv2.Rodrigues(q.rotation_matrix)[0].flatten() #* np.array([1, 1, 1])
    
    # get marker
    marker = dict_to_np(snapshot.Markers) 
    
    # create person device
    person = Device(name='person', translation=translation.tolist(), rotation=rotation.tolist())
    
    # get eye centers
    left_eye_center = dict_to_np(snapshot.GazeEstimation['eyeSphereCenterLeft']) * flip_array
    right_eye_center = dict_to_np(snapshot.GazeEstimation['eyeSphereCenterRight']) * flip_array
    
    ### append data ###
    
    # pupils
    left_pupil_centers.append(left_pupil_center.flatten())
    right_pupil_centers.append(right_pupil_center.flatten())
    
    # head position
    rotations.append(rotation)
    translations.append(translation)
    
    # markers
    markers.append(marker.flatten())
    
    # face points
    face_points_array.append(face_points)
    
    # eye centers
    left_eye_centers.append(left_eye_center.flatten())
    right_eye_centers.append(right_eye_center.flatten())
    
    # face vector for visualization
    faceGaze = np.array([translation.flatten(), (translation - 4 * rotation).flatten()])
    
left_pupil_centers = np.array(left_pupil_centers)
right_pupil_centers = np.array(right_pupil_centers)

rotations = np.array(rotations)
translations = np.array(translations)
markers = np.array(markers)

face_points_array = np.array(face_points_array)

left_eye_centers = np.array(left_eye_centers)
right_eye_centers = np.array(right_eye_centers)

100%|██████████| 40/40 [00:08<00:00,  4.88it/s]


In [9]:
right_gazes_true = markers - right_pupil_centers
markers_self = vectors_to(markers, rotations, translations)

right_pupil_self = vectors_to(right_pupil_centers, rotations, translations, to='self')
left_pupil_self = vectors_to(left_pupil_centers, rotations, translations, to='self')

right_gazes_true = markers - right_pupil_centers
right_gazes_true = right_gazes_true / np.linalg.norm(right_gazes_true, axis=1, keepdims=True)

left_gazes_true = markers - left_pupil_centers
left_gazes_true = left_gazes_true / np.linalg.norm(left_gazes_true, axis=1, keepdims=True)

right_gazes_true_person = markers_self - right_pupil_self
right_gazes_true_person = right_gazes_true_person / np.linalg.norm(right_gazes_true_person, axis=1, keepdims=True)

left_gazes_true_person = markers_self - left_pupil_self
left_gazes_true_person = left_gazes_true_person / np.linalg.norm(left_gazes_true_person, axis=1, keepdims=True)

In [10]:
def create_trace_3d(data, **kwargs):
    
    assert data.ndim == 2
    assert data.shape[1] == 3
    
    x, y, z = data.T
    return go.Scatter3d(x=x, y=z, z=-y, **kwargs)

marker_style={
    'mode': 'markers',
    'marker': dict(size=3, line=dict(color='rgba(217, 217, 217, 0.14)', width=0.5), opacity=0.8)
}

line_style={
#     'mode': 'line',
    'marker': dict(size=3, line=dict(color='rgba(217, 217, 217, 0.14)', width=0.5), opacity=0.8)
}

In [11]:
def cost(x, pt2, pt1):
    diff1 = pt2 - pt1
    diff1 = diff1 / np.linalg.norm(diff1, axis=1, keepdims=True)
    diff2 = pt1 - x.reshape(1, 3)
    diff2_norm = np.linalg.norm(diff2, axis=1, keepdims=True)
    diff2 = diff2 / diff2_norm
#     print(diff2_norm)
    return (((diff1 - diff2) ** 2) + 1 * (diff2_norm ** 2)).sum()

right_pupil_centers_person = vectors_to(right_pupil_centers, rotations, translations, 'self')
markers_person = vectors_to(markers, rotations, translations, 'self')
minimized = minimize(cost, right_pupil_centers_person.mean(axis=0), args=(markers_person, right_pupil_centers_person))
print(minimized)
eye_centers_origin = vectors_to(np.tile(minimized.x, (40, 1)), rotations, translations, 'origin')

      fun: 2.2278601942629708
 hess_inv: array([[6.46068254e-05, 8.99811994e-06, 1.01101750e-05],
       [8.99811994e-06, 1.02712078e-04, 9.75473308e-05],
       [1.01101750e-05, 9.75473308e-05, 2.55775727e-04]])
      jac: array([1.89542770e-05, 2.23815441e-05, 1.44243240e-05])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 452
      nit: 17
     njev: 88
   status: 2
  success: False
        x: array([-0.03671895, -0.01583344, -0.01074297])


In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import FunctionTransformer

# X_right = vectors_to(right_pupil_centers, rotations, translations)
# Y_right = vectors_to(right_gazes_true+right_pupil_centers, rotations, translations) - X_right
# Y_right = vecs2angles(Y_right / np.linalg.norm(Y_right, axis=1, keepdims=True))

X_right = np.column_stack((right_pupil_centers, right_eye_centers, rotations, translations))
Y_right = vecs2angles(right_gazes_true)

# X_left = vectors_to(left_pupil_centers, rotations, translations)
# Y_left = vectors_to(left_gazes_true+left_pupil_centers, rotations, translations) - X_left
# Y_left = vecs2angles(Y_left / np.linalg.norm(Y_left, axis=1, keepdims=True))

X_left = np.column_stack((left_pupil_centers, left_eye_centers, rotations, translations))
Y_left = vecs2angles(left_gazes_true)

poly_features = 3
alpha = 1e-5
pipe_right = make_pipeline(PolynomialFeatures(poly_features), Ridge(alpha))
pipe_left = make_pipeline(PolynomialFeatures(poly_features), Ridge(alpha))
kf = KFold(40)


left_angle_errs = []
right_angle_errs = []
for train_index, test_index in kf.split(X_left):
    
    # fit
    pipe_right.fit(X_right[train_index], Y_right[train_index])
    pipe_left.fit(X_left[train_index], Y_left[train_index])
        
    # test
    Y_pred_right = pipe_right.predict(X_right[test_index])
    Y_pred_left = pipe_left.predict(X_left[test_index])    
    
    left_angle_err = calc_angle_spherical(Y_pred_left, Y_left[test_index])
    right_angle_err = calc_angle_spherical(Y_pred_right, Y_right[test_index])
    
    left_angle_errs.append(left_angle_err)
    right_angle_errs.append(right_angle_err)
    print('===========')
    print(f'Test point {test_index % 8 + 1} \n\tleft: {left_angle_err},\n\tright: {right_angle_err}')
    
left_angle_errs = np.array(left_angle_errs)
right_angle_errs = np.array(right_angle_errs)

print(left_angle_errs.mean(), right_angle_errs.mean())

Test point [1] 
	left: [9.05426856],
	right: [6.0052827]
Test point [2] 
	left: [3.75967668],
	right: [8.21440855]
Test point [3] 
	left: [8.77035873],
	right: [7.30884079]
Test point [4] 
	left: [5.66382572],
	right: [2.70021972]
Test point [5] 
	left: [2.54688127],
	right: [7.34339501]
Test point [6] 
	left: [5.17026377],
	right: [6.60610084]
Test point [7] 
	left: [4.39560514],
	right: [5.51234005]
Test point [8] 
	left: [2.98996727],
	right: [3.81908063]
Test point [1] 
	left: [5.39675429],
	right: [13.2205824]
Test point [2] 
	left: [5.91057519],
	right: [4.85343299]
Test point [3] 
	left: [4.97922704],
	right: [2.02913428]
Test point [4] 
	left: [4.47683876],
	right: [6.25220126]
Test point [5] 
	left: [14.28010546],
	right: [16.20864607]
Test point [6] 
	left: [5.23077551],
	right: [4.90471986]
Test point [7] 
	left: [1.09186166],
	right: [3.91495188]
Test point [8] 
	left: [2.57005073],
	right: [7.3439074]
Test point [1] 
	left: [6.14824098],
	right: [13.50338985]
Test point [2

In [56]:
pipe_right.fit(X_right, Y_right)
pipe_left.fit(X_left, Y_left)

right_gazes_pred = angles2vecs(pipe_right.predict(X_right))
left_gazes_pred = angles2vecs(pipe_left.predict(X_left))

# right_gazes_pred = vectors_to(angles2vecs(pipe_right.predict(X_right)), rotations, translations, to='origin')
# left_gazes_pred = vectors_to(angles2vecs(pipe_left.predict(X_left)), rotations, translations, to='origin')

In [60]:
i = 5

point_traces = [
    create_trace_3d(data.reshape(-1, 3), **marker_style) 
    for data in [translations[i],
#                  markers,
                 np.stack((left_pupil_centers[i], right_pupil_centers[i])),
                 face_points_array[i]]
]

line_left_traces = [
    create_trace_3d(np.stack((left_pupil_centers[i], left_pupil_centers[i]+left_gazes_true[i]/10)), **line_style)
]
line_right_traces = [
    create_trace_3d(np.stack((right_pupil_centers[i], right_pupil_centers[i]+right_gazes_true[i]/10)), **line_style)
]

line_pred_left_traces = [
    create_trace_3d(np.stack((left_pupil_centers[i], left_pupil_centers[i]+left_gazes_pred[i]/10)), **line_style)
]
line_pred_right_traces = [
    create_trace_3d(np.stack((right_pupil_centers[i], right_pupil_centers[i]+right_gazes_pred[i]/10)), **line_style)
]

layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0), width=1000, height=1000)

fig = go.Figure(data=point_traces+line_left_traces+line_right_traces+line_pred_left_traces+line_pred_right_traces,
                layout=layout)
iplot(fig, filename='simple-3d-scatter')